# Using 🤗 PEFT & bitsandbytes to finetune a LoRa checkpoint




In [1]:
# !pip install -q bitsandbytes datasets accelerate loralib
# !pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 49.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
!nvidia-smi -L

GPU 0: NVIDIA RTX 6000 Ada Generation (UUID: GPU-acb2d5c3-80e3-44af-4af5-ebf49de440ea)
GPU 1: NVIDIA RTX 6000 Ada Generation (UUID: GPU-9414d4d6-2f92-8210-9fe5-ad990c9464f7)


In [4]:
import torch 
print(torch.cuda.is_available())

True


In [5]:
device_index = 0  # Replace with the index of the device you want to use
torch.cuda.set_device(device_index)

In [2]:
print("Current CUDA device index:", torch.cuda.current_device())
print("Current CUDA device name:", torch.cuda.get_device_name(torch.cuda.current_device()))

Current CUDA device index: 0
Current CUDA device name: NVIDIA RTX 6000 Ada Generation


### Setup the model

In [10]:
import os

import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

# model = AutoModelForCausalLM.from_pretrained(
#     "bigscience/bloom-7b1",
#     load_in_8bit=True,
#     device_map="cuda:0"
# )

# tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-7b1")

# Load model and tokenizer locally
model = AutoModelForCausalLM.from_pretrained(
    "/data/8tb/hf_home/hub/models--bigscience--bloom-7b1/snapshots/e83e90ba86f87f74aa2731cdab25ccf33976bd66",
    load_in_8bit=True,
    device_map="cuda:0" # IMPORTANT - let the model use the GPU
)

tokenizer = AutoTokenizer.from_pretrained("/data/8tb/hf_home/hub/models--bigscience--bloom-7b1/snapshots/e83e90ba86f87f74aa2731cdab25ccf33976bd66")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Freezing the original weights


In [11]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

### Setting up the LoRa Adapters

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [13]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=16, # attention heads
    lora_alpha=32, # alpha scaling
    # target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 7864320 || all params: 7076880384 || trainable%: 0.11112693126452029


## Data

In [14]:
import transformers
from datasets import load_dataset
data = load_dataset("Abirate/english_quotes") # Size is 2.5k rows


In [15]:
def merge_columns(example):
    example["prediction"] = example["quote"] + " ->: " + str(example["tags"])
    return example

data['train'] = data['train'].map(merge_columns)
data['train']["prediction"][:5]

["“Be yourself; everyone else is already taken.” ->: ['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']",
 "“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.” ->: ['best', 'life', 'love', 'mistakes', 'out-of-control', 'truth', 'worst']",
 "“Two things are infinite: the universe and human stupidity; and I'm not sure about the universe.” ->: ['human-nature', 'humor', 'infinity', 'philosophy', 'science', 'stupidity', 'universe']",
 "“So many books, so little time.” ->: ['books', 'humor']",
 "“A room without books is like a body without a soul.” ->: ['books', 'simile', 'soul']"]

In [16]:
data['train'][0]

{'quote': '“Be yourself; everyone else is already taken.”',
 'author': 'Oscar Wilde',
 'tags': ['be-yourself',
  'gilbert-perreira',
  'honesty',
  'inspirational',
  'misattributed-oscar-wilde',
  'quote-investigator'],
 'prediction': "“Be yourself; everyone else is already taken.” ->: ['be-yourself', 'gilbert-perreira', 'honesty', 'inspirational', 'misattributed-oscar-wilde', 'quote-investigator']"}

In [17]:
data = data.map(lambda samples: tokenizer(samples['prediction']), batched=True)

Map:   0%|          | 0/2508 [00:00<?, ? examples/s]

In [18]:
data

DatasetDict({
    train: Dataset({
        features: ['quote', 'author', 'tags', 'prediction', 'input_ids', 'attention_mask'],
        num_rows: 2508
    })
})

In [19]:
print("Current CUDA device index:", torch.cuda.current_device())

Current CUDA device index: 0


### Training

In [20]:
# os.environ["CUDA_VISIBLE_DEVICES"] = "1"

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=16,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
        num_train_epochs=3
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a BloomTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/yushi/anaconda3/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss
1,3.310700
2,3.246600
3,3.520000
4,3.148600
5,3.302300
6,3.135000
7,3.454500
8,3.231100
9,3.508700
10,3.300600


TrainOutput(global_step=200, training_loss=2.272856888771057, metrics={'train_runtime': 1053.8576, 'train_samples_per_second': 6.073, 'train_steps_per_second': 0.19, 'total_flos': 1.898110336922419e+16, 'train_loss': 2.272856888771057, 'epoch': 2.55})

## Share adapters on the 🤗 Hub

In [22]:
# Save the model locally
trainer.save_model("/data/8tb/hf_home/hub")

In [5]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

peft_model_id = "/data/8tb/hf_home/hub/models--yy0514--bloom-7b1-lora-try/snapshots/d03e637fff7a1f880e0d201db5a8342a481faa4a/"
config = PeftConfig.from_pretrained(peft_model_id)
# print(config)

model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map="auto") 
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# Load the Lora model
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
# model.push_to_hub("yy0514/bloom-7b1-lora-try-2",
#                   use_auth_token=True,
#                   commit_message="Try basic lora training in VS code, 200 epochs",
#                   private=True)

## Load adapters from the Hub

In [3]:
# import torch
# from peft import PeftModel, PeftConfig
# from transformers import AutoModelForCausalLM, AutoTokenizer

# peft_model_id = "yy0514/bloom-7b1-lora-try"
# config = PeftConfig.from_pretrained(peft_model_id)
# # print(config)

# model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, return_dict=True, load_in_8bit=True, device_map='cuda:0') 
# tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

# # Load the Lora model
# model = PeftModel.from_pretrained(model, peft_model_id)

## Inference

In [4]:
batch = tokenizer("“Training models with PEFT and LoRa is cool” ->: ", return_tensors='pt')

with torch.cuda.amp.autocast():
  output_tokens = model.generate(**batch, max_new_tokens=50)

print('\n\n', tokenizer.decode(output_tokens[0], skip_special_tokens=True))

/home/yushi/anaconda3/lib/python3.11/site-packages/transformers/generation/utils.py:1671: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
/home/yushi/anaconda3/lib/python3.11/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")




 “Training models with PEFT and LoRa is cool” ->:  ['training', 'teaching'] ->: ['training', 'teaching'] ->: ['training', 'teaching'] ->: ['training', 'teaching'] ->: ['training', 'teaching'] ->:
